In [ ]:
#! pip install azure-search-documents --pre
#! pip install openai
#! pip install python-dotenv
#! pip install --upgrade azure-search-documents

In [9]:
# Import required libraries  
import os  
import json  
import openai  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SemanticSettings,  
    VectorSearch, 
    HnswVectorSearchAlgorithmConfiguration,  
)  

from langchain.document_loaders import AzureBlobStorageContainerLoader
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.document_loaders import AzureBlobStorageContainerLoader
from langchain.text_splitter import CharacterTextSplitter
from azure.storage.blob import BlobServiceClient
import tempfile
from langchain.document_loaders import PyPDFLoader
from azure.core.exceptions import HttpResponseError 
from langchain.embeddings.openai import OpenAIEmbeddings

In [ ]:
# Import required libraries  
import os  
import json  
import openai  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SemanticSettings,  
    VectorSearch, 
    HnswVectorSearchAlgorithmConfiguration,  
)  

from langchain.document_loaders import AzureBlobStorageContainerLoader
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.document_loaders import AzureBlobStorageContainerLoader
from langchain.text_splitter import CharacterTextSplitter
from azure.storage.blob import BlobServiceClient
import tempfile
from langchain.document_loaders import PyPDFLoader
from azure.core.exceptions import HttpResponseError 
from langchain.embeddings.openai import OpenAIEmbeddings
# Configure environment variables  
load_dotenv()  
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") 
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
openai.api_type = "azure"  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")  
credential = AzureKeyCredential("jA50VnyT0OAeuHpY0RxJGz7z9kfrmdj7YF5PHYiPXjAzSeBS6hJg")
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://testingchat.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "e8143eabb02541259054426630db12a7"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
model: str = "text-embedding-ada-002"
#pdf_file_name = "Alaska Airlines and Horizon Air SMS Manual.pdf"     

AZURE_SEARCH_SERVICE_ENDPOINT = "https://genaisafety.search.windows.net"
AZURE_SEARCH_ADMIN_KEY = "jA50VnyT0OAeuHpY0RxJGz7z9kfrmdj7YF5PHYiPXjAzSeBS6hJg"
def pdf_text_extraction(pdf_file_path):
    if pdf_file_path is not None:
        with open(pdf_file_path, 'rb') as pdf_file:
            # Create a temporary file to write the PDF contents
            with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
                tmp_file.write(pdf_file.read())
                pdf_path = tmp_file.name

        # Now you can use the PyPDFLoader on the temporary PDF file
        loader = PyPDFLoader(pdf_path)
        pages_split = loader.load_and_split()
    return pages_split
def create_index(AZURE_SEARCH_INDEX_NAME, fields):
    vector_search = VectorSearch(
        algorithm_configurations=[
            HnswVectorSearchAlgorithmConfiguration(
                name="my-vector-config",
                kind="hnsw",
                parameters={
                    "m": 4,
                    "efConstruction": 400,
                    "efSearch": 500,
                    "metric": "cosine"
                }
            )
        ]
    )


    semantic_config = SemanticConfiguration(
        name="my-semantic-config",
        prioritized_fields=PrioritizedFields(
            title_field=SemanticField(field_name="id"),
            prioritized_keywords_fields=[SemanticField(field_name="content")]
        )
    )

    # Create the semantic settings with the configuration
    semantic_settings = SemanticSettings(configurations=[semantic_config])

    # Create the search index with the semantic settings
    index = SearchIndex(name=AZURE_SEARCH_INDEX_NAME, fields=fields,
                        vector_search=vector_search, semantic_settings=semantic_settings)

    result = search_client.create_or_update_index(index)
    return result
embeddings = OpenAIEmbeddings(deployment = model,chunk_size=1) 

# Replace with your actual connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=storageaccountgenai;AccountKey=WAfyL4uqWgarFdx1ibNgWv9lmOINODLuN6nnLSQLgE/iuHhKGi1pYd6NQJ6LBnZO/DnnQfhbNSWi+AStsOLf1Q==;EndpointSuffix=core.windows.net"

# Replace with your container name
container_name = "safety"

# Create a BlobServiceClient using the connection string
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get a reference to the container
container_client = blob_service_client.get_container_client(container_name)
# List blobs in the container
blobs = container_client.list_blobs()
dict_data = []
for blob in blobs:
    if blob.name.startswith("manuals") and blob.name.endswith(".pdf"): #and ("manuals/Alaska Airlines and Horizon Air SMS Manual.pdf" in blob.name):  # Check if the blob is a PDF file
        #blob_client = container_client.get_blob_client(blob)
        pdf_file_name=str(blob.name).split("/")[1] 
        bid=str(blob.name).split("/")[1].split(".")[0].lower().replace(" ","_").replace("_","-")
        print(blob.name)
        
                      
        documents = pdf_text_extraction(pdf_file_name)

        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
        docs = text_splitter.split_documents(documents)
        print("PDF_NAME, length", blob.name, len(docs))
        
        
            
        for index,doc in enumerate(docs):
            dict_data.append({
                'id': bid+'_'+ str(index),
                'content': doc.page_content,  # You can set a title based on your needs
                'page_number': doc.metadata['page'],
                'metadata': doc.metadata,
            })

        print(f"splitted into {len(docs)} documents") 
    
        

        for item in dict_data:
            title = item['id']
            content = item['content']
            title_embeddings = embeddings.embed_query(title)
            content_embeddings = embeddings.embed_query(content)
            item['titleVector'] = title_embeddings
            item['content_vector'] = content_embeddings
            
        bid=str(blob.name).split("/")[1].split(".")[0].lower().replace(" ","_").replace("_","-")
        AZURE_SEARCH_INDEX_NAME=bid + "-index"
        print(AZURE_SEARCH_INDEX_NAME)

        
        search_client = SearchIndexClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, credential=credential)
        search_client_v2 = SearchClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, index_name=AZURE_SEARCH_INDEX_NAME, credential=credential)

        
        # Define your Azure Cognitive Search credentials and index settings as previously done
        # Check if the index already exists
        # existing_index = None
        # Define your Azure Cognitive Search credentials and index settings as previously done

        combined_content = ''.join([p.page_content for p in docs])
        combined_vector = embeddings.embed_query(combined_content)
        try:
            existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
        except Exception as e:
            existing_index == None
            # The index doesn't exist, so create a new one
        if existing_index == None:
            fields = [
            SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
            SearchableField(name="title", type=SearchFieldDataType.String),
            SearchableField(name="content", type=SearchFieldDataType.String),
            SearchableField(name="combined_content", type=SearchFieldDataType.String),
            SearchableField(name="page_number", type=SearchFieldDataType.String),
            SearchableField(name="metadata", type=SearchFieldDataType.String, searchable=True),
            SearchField(name="combined_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                        searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
            SearchField(name="content_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                        searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
            ]

            result = create_index(AZURE_SEARCH_INDEX_NAME, fields)
            print(f' {result.name} created index')                        
            
            final_doc=[]
            for index,doc in enumerate(docs):
                final_doc.append({'id':dict_data[index]['id'], 'title':dict_data[index]['id'],
                                  'content':dict_data[index]['content'],
                                  'combined_content':combined_content,
                                  'page_number':str(dict_data[index]['page_number']), 
                                  'metadata': json.dumps(dict_data[index]['metadata']),
                                  'combined_vector': combined_vector, 'content_vector':dict_data[index]['content_vector']})
                
            # Define a list to store batches of documents
            document_batches = [final_doc[i] for i in range(0, len(final_doc))]

            # Upload documents in batches
            for batch in document_batches:
                try:
                    result = search_client_v2.upload_documents(batch)
                except HttpResponseError as e:
                    print(f"Error uploading documents: {e}")    

            print("Vector store added successfully")

        else:
            try:
                # Check if the index exists
                # existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
                # print(f"Index '{AZURE_SEARCH_INDEX_NAME}' already exists. You can use the existing index.")

                final_doc=[]
                metadata_str = json.dumps(dict_data[index]['metadata'])

                for index,doc in enumerate(docs):
                    final_doc.append({'id':dict_data[index]['id'], 'title':dict_data[index]['id'],
                                      'content':dict_data[index]['content'],
                                      'combined_content':combined_content,
                                      'page_number':str(dict_data[index]['page_number']), 
                                      'metadata': metadata_str,
                                      'combined_vector':combined_vector, 'content_vector':dict_data[index]['content_vector']})

                # Define a list to store batches of documents
                document_batches = [final_doc[i] for i in range(0, len(final_doc))]

                # Upload documents in batches
                for batch in document_batches:
                    try:
                        result = search_client_v2.upload_documents(batch)
                    except HttpResponseError as e:
                        print(f"Error uploading documents: {e}")    

                print("Vector store added successfully")
            

            except Exception as e:
                print(e)
                pass
    else:
        continue

In [14]:
# Configure environment variables  
load_dotenv()  
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") 
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
openai.api_type = "azure"  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")  
credential = AzureKeyCredential("jA50VnyT0OAeuHpY0RxJGz7z9kfrmdj7YF5PHYiPXjAzSeBS6hJg")

In [11]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://testingchat.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "e8143eabb02541259054426630db12a7"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
model: str = "text-embedding-ada-002"
#pdf_file_name = "Alaska Airlines and Horizon Air SMS Manual.pdf"     

AZURE_SEARCH_SERVICE_ENDPOINT = "https://genaisafety.search.windows.net"
AZURE_SEARCH_ADMIN_KEY = "jA50VnyT0OAeuHpY0RxJGz7z9kfrmdj7YF5PHYiPXjAzSeBS6hJg"

In [4]:
def pdf_text_extraction(pdf_file_path):
    if pdf_file_path is not None:
        with open(pdf_file_path, 'rb') as pdf_file:
            # Create a temporary file to write the PDF contents
            with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
                tmp_file.write(pdf_file.read())
                pdf_path = tmp_file.name

        # Now you can use the PyPDFLoader on the temporary PDF file
        loader = PyPDFLoader(pdf_path)
        pages_split = loader.load_and_split()
    return pages_split

In [8]:
def create_index(AZURE_SEARCH_INDEX_NAME, fields):
    vector_search = VectorSearch(
        algorithm_configurations=[
            HnswVectorSearchAlgorithmConfiguration(
                name="my-vector-config",
                kind="hnsw",
                parameters={
                    "m": 4,
                    "efConstruction": 400,
                    "efSearch": 500,
                    "metric": "cosine"
                }
            )
        ]
    )


    semantic_config = SemanticConfiguration(
        name="my-semantic-config",
        prioritized_fields=PrioritizedFields(
            title_field=SemanticField(field_name="id"),
            prioritized_keywords_fields=[SemanticField(field_name="content")]
        )
    )

    # Create the semantic settings with the configuration
    semantic_settings = SemanticSettings(configurations=[semantic_config])

    # Create the search index with the semantic settings
    index = SearchIndex(name=AZURE_SEARCH_INDEX_NAME, fields=fields,
                        vector_search=vector_search, semantic_settings=semantic_settings)

    result = search_client.create_or_update_index(index)
    return result

In [ ]:
embeddings = OpenAIEmbeddings(deployment = model,chunk_size=1) 

# Replace with your actual connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=storageaccountgenai;AccountKey=WAfyL4uqWgarFdx1ibNgWv9lmOINODLuN6nnLSQLgE/iuHhKGi1pYd6NQJ6LBnZO/DnnQfhbNSWi+AStsOLf1Q==;EndpointSuffix=core.windows.net"

# Replace with your container name
container_name = "safety"

# Create a BlobServiceClient using the connection string
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get a reference to the container
container_client = blob_service_client.get_container_client(container_name)

In [51]:
# List blobs in the container
blobs = container_client.list_blobs()
dict_data = []
for blob in blobs:
    if blob.name.startswith("manuals") and blob.name.endswith(".pdf"): #and ("manuals/Alaska Airlines and Horizon Air SMS Manual.pdf" in blob.name):  # Check if the blob is a PDF file
        #blob_client = container_client.get_blob_client(blob)
        pdf_file_name=str(blob.name).split("/")[1] 
        bid=str(blob.name).split("/")[1].split(".")[0].lower().replace(" ","_").replace("_","-")
        print(blob.name)
        
                      
        documents = pdf_text_extraction(pdf_file_name)

        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
        docs = text_splitter.split_documents(documents)
        print("PDF_NAME, length", blob.name, len(docs))
        
        
            
        for index,doc in enumerate(docs):
            dict_data.append({
                'id': bid+'_'+ str(index),
                'content': doc.page_content,  # You can set a title based on your needs
                'page_number': doc.metadata['page'],
                'metadata': doc.metadata,
            })

        print(f"splitted into {len(docs)} documents") 
    
        

        for item in dict_data:
            title = item['id']
            content = item['content']
            title_embeddings = embeddings.embed_query(title)
            content_embeddings = embeddings.embed_query(content)
            item['titleVector'] = title_embeddings
            item['content_vector'] = content_embeddings
            
        bid=str(blob.name).split("/")[1].split(".")[0].lower().replace(" ","_").replace("_","-")
        AZURE_SEARCH_INDEX_NAME=bid + "-index"
        print(AZURE_SEARCH_INDEX_NAME)

        
        search_client = SearchIndexClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, credential=credential)
        search_client_v2 = SearchClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, index_name=AZURE_SEARCH_INDEX_NAME, credential=credential)

        
        # Define your Azure Cognitive Search credentials and index settings as previously done
        # Check if the index already exists
        # existing_index = None
        # Define your Azure Cognitive Search credentials and index settings as previously done

        combined_content = ''.join([p.page_content for p in docs])
        combined_vector = embeddings.embed_query(combined_content)
        try:
            existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
        except Exception as e:
            existing_index == None
            # The index doesn't exist, so create a new one
        if existing_index == None:
            fields = [
            SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
            SearchableField(name="title", type=SearchFieldDataType.String),
            SearchableField(name="content", type=SearchFieldDataType.String),
            SearchableField(name="combined_content", type=SearchFieldDataType.String),
            SearchableField(name="page_number", type=SearchFieldDataType.String),
            SearchableField(name="metadata", type=SearchFieldDataType.String, searchable=True),
            SearchField(name="combined_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                        searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
            SearchField(name="content_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                        searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
            ]

            result = create_index(AZURE_SEARCH_INDEX_NAME, fields)
            print(f' {result.name} created index')                        
            
            final_doc=[]
            for index,doc in enumerate(docs):
                final_doc.append({'id':dict_data[index]['id'], 'title':dict_data[index]['id'],
                                  'content':dict_data[index]['content'],
                                  'combined_content':combined_content,
                                  'page_number':str(dict_data[index]['page_number']), 
                                  'metadata': json.dumps(dict_data[index]['metadata']),
                                  'combined_vector': combined_vector, 'content_vector':dict_data[index]['content_vector']})
                
            # Define a list to store batches of documents
            document_batches = [final_doc[i] for i in range(0, len(final_doc))]

            # Upload documents in batches
            for batch in document_batches:
                try:
                    result = search_client_v2.upload_documents(batch)
                except HttpResponseError as e:
                    print(f"Error uploading documents: {e}")    

            print("Vector store added successfully")

        else:
            try:
                # Check if the index exists
                # existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
                # print(f"Index '{AZURE_SEARCH_INDEX_NAME}' already exists. You can use the existing index.")

                final_doc=[]
                metadata_str = json.dumps(dict_data[index]['metadata'])

                for index,doc in enumerate(docs):
                    final_doc.append({'id':dict_data[index]['id'], 'title':dict_data[index]['id'],
                                      'content':dict_data[index]['content'],
                                      'combined_content':combined_content,
                                      'page_number':str(dict_data[index]['page_number']), 
                                      'metadata': metadata_str,
                                      'combined_vector':combined_vector, 'content_vector':dict_data[index]['content_vector']})

                # Define a list to store batches of documents
                document_batches = [final_doc[i] for i in range(0, len(final_doc))]

                # Upload documents in batches
                for batch in document_batches:
                    try:
                        result = search_client_v2.upload_documents(batch)
                    except HttpResponseError as e:
                        print(f"Error uploading documents: {e}")    

                print("Vector store added successfully")
            

            except Exception as e:
                print(e)
                pass
    else:
        continue

manuals/AAG_Safety_Manual.pdf
PDF_NAME, length manuals/AAG_Safety_Manual.pdf 332
splitted into 332 documents
aag-safety-manual-index
 aag-safety-manual-index created index


KeyboardInterrupt: 

In [57]:
#fetch table of content page
print(final_doc[1]['content'])

{'id': 'aag-safety-manual_1', 'title': 'aag-safety-manual_1', 'content': 'i of iii\nRevision HighlightsSafety Manual\nUncontrolled copy when downloaded or printed from an electronic document.\nRevision Highlights\nChanges inserted with this revision are listed in the table below.\nChapter Description\nAllUpdated links throughout the Safety Manual to reflect new locations of sections as listed below\nReduced instances of wasted space between Sections\nTOCChanged Chapter 4 title from “Incident, Accident, and Ground Damage Investigation and \nReporting” to “Ground Damage and NTSB Reporting”\nDeleted Section 4.1 - Overview (Content moved and edited into Chapter 9 of the SMS Manual)\nDeleted Section 4.2 - Safety Hotline (Content moved and edited into Chapter 9 of the SMS \nManual)\nDeleted Section 4.3 - Safety Hazard Reporting (Content moved and edited into Chapter 9 of the \nSMS Manual)\nDeleted Section 4.4 - Operational Irregularity Reports (Content moved and edited into Chapter 9 of \nth

In [28]:
page_number = 18
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
view = documents[page_number-1]
page_result = text_splitter.split_text(view.page_content)
print(page_result)

["1 of 2\nChapter 1Safety Management System Manual\nUncontrolled copy when downloaded or printed from an electronic document.\nChapter 1: Safety Policy\n1.1 Introduction\nA. The Safety Policy reflects the Alaska Airlines' and Horizon Air's commitment to safety. It was \ndeveloped in accordance with all applicable regulatory requirements in 14 CFR Part 5. The \nSafety Policy is signed by the Accountable Executives who will ensure the information is \ncommunicated and adhered to throughout the organization. A more in depth review of the policy \nis in Chapter 7 .Rev. 18\n6/16/2023"]


In [29]:
start_page = 18
end_page = 20
texts = []
for page_num in (start_page, end_page+1):
    view = documents[page_number-1]
    page_texts =  text_splitter.split_text(view.page_content)
    texts.extend(page_texts)
print(texts)

["1 of 2\nChapter 1Safety Management System Manual\nUncontrolled copy when downloaded or printed from an electronic document.\nChapter 1: Safety Policy\n1.1 Introduction\nA. The Safety Policy reflects the Alaska Airlines' and Horizon Air's commitment to safety. It was \ndeveloped in accordance with all applicable regulatory requirements in 14 CFR Part 5. The \nSafety Policy is signed by the Accountable Executives who will ensure the information is \ncommunicated and adhered to throughout the organization. A more in depth review of the policy \nis in Chapter 7 .Rev. 18\n6/16/2023", "1 of 2\nChapter 1Safety Management System Manual\nUncontrolled copy when downloaded or printed from an electronic document.\nChapter 1: Safety Policy\n1.1 Introduction\nA. The Safety Policy reflects the Alaska Airlines' and Horizon Air's commitment to safety. It was \ndeveloped in accordance with all applicable regulatory requirements in 14 CFR Part 5. The \nSafety Policy is signed by the Accountable Executi

In [31]:
# Pure Vector Search
query = "What is the definition of a Minor Accident?"

query_embeddings = embeddings.embed_query(query)
#combined_vector = embeddings.embed_query(combined_content)
# Convert the query embeddings to a list
#query_embeddings_list = query_embeddings.tolist()
 
#search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector = Vector(value= query_embeddings, k=3, fields="combined_vector")
  
results = search_client_v2.search(  
    search_text=None,  
    vectors= [vector],
    select=["id" ,"combined_content"],
)  
  
# for result in results:  
#     print(f"id: {result['id']}")  
#     print(f"Score: {result['@search.score']}")  
#     print(f"combined_content: {result['combined_content']}") 
for result in results:
    print(f"combined_content: {result['combined_content']}") 

combined_content: Uncontrolled copy when downloaded or printed from an electronic document.
Safety Management System 
(SMS) Manual
Alaska Airlines, Inc. / Horizon Air
(Alaska Air Group)
Revision 18 6/16/2023Safety Management System Manual
Uncontrolled copy when downloaded or printed from an electronic document.
Intentionally Left Blanki of vi
Revision HighlightsSafety Management System Manual
Uncontrolled copy when downloaded or printed from an electronic document.
Revision Highlights
Changes inserted with this revision are listed in the table below.
Table 1-1: 
Chapter Description
All Changed references of “risk matrix” to “SMS Matrix”
Corrected minor spelling and grammatical errors
Fixed minor innacuracies
Changed instances of “corrective action” and “preventive/
corrective action” to “corrective/preventive action” where 
applicable
1 1.1.B - Updated the Safety Policy to improve readability, replace 
“customers” with “guests”, and include business partners in lines 3 
and 6
2 2.2.D -

In [79]:
# Pure Vector Search
query = "What is the definition of a Minor Accident?"

query_embeddings = embeddings.embed_query(query)

# Convert the query embeddings to a list
#query_embeddings_list = query_embeddings.tolist()
 
#search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector = Vector(value= query_embeddings, k=3, fields="content_vector")
  
results = search_client_v2.search(  
    search_text=None,  
    vectors= [vector],
    select=["id" ,"content", "page_number"],
)  
  
for result in results:  
    print(f"id: {result['id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"content: {result['content']}") 
    print(f"page_number: {result['page_number']}\n") 
    

id: aag-safety-manual_70
Score: 0.83731794
 combined_content: Uncontrolled copy when downloaded or printed from an electronic document.
Safety Management System 
(SMS) Manual
Alaska Airlines, Inc. / Horizon Air
(Alaska Air Group)
Revision 18 6/16/2023Safety Management System Manual
Uncontrolled copy when downloaded or printed from an electronic document.
Intentionally Left Blanki of vi
Revision HighlightsSafety Management System Manual
Uncontrolled copy when downloaded or printed from an electronic document.
Revision Highlights
Changes inserted with this revision are listed in the table below.
Table 1-1: 
Chapter Description
All Changed references of “risk matrix” to “SMS Matrix”
Corrected minor spelling and grammatical errors
Fixed minor innacuracies
Changed instances of “corrective action” and “preventive/
corrective action” to “corrective/preventive action” where 
applicable
1 1.1.B - Updated the Safety Policy to improve readability, replace 
“customers” with “guests”, and include b

In [32]:
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    azure_search_key=AZURE_SEARCH_ADMIN_KEY,
    index_name = AZURE_SEARCH_INDEX_NAME,
    embedding_function=embeddings.embed_query,
)
    
# Perform a similarity search
docs_vector = vector_store.similarity_search(
    query="What is the definition of a Minor Accident?",
    k=3,
    search_type="similarity",
)
print(docs_vector[0].page_content)

8 of 9
Chapter 4Safety Manual
Uncontrolled copy when downloaded or printed from an electronic document.
4. Ground Support Equipment: Property damage to GSE or caused by GSE. Also, personnel 
injury or illness caused by GSE. If GSE is involved in an aircraft accident or incident, report 
under that category as well.
5. Non-Employee: Occurrence involving injury or illness to either a customer or non-customer. 
If injury is sustained and involves Company aircraft, facilities, or GSE, a report under that 
category should be submitted.
4.5.1 Classification for Notification and Reporting
A. Major Accident: An accident which results in significant property damage, serious injury or 
illness which requires immediate emergency medical care, or results in death. For aircraft 
accidents, the term substantial damage is per the definition in the NTSB Part 830 Rules.
B. Minor Accident: An accident that results in significant property damage, or results in injury or 
illness requiring medical care.
C

In [65]:
#!pip install pyodbc azure-storage-blob

In [5]:
import pyodbc

# Replace with your own database connection details
server = 'ORA_Safety.sql.insideaag.com,1450'
database = 'Safety'
username = 'SafetyME'
password = 'Shakennotstirred1!'

connection_string = f'DRIVER=ODBC Driver 13 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

#isql -v 'ORA_Safety.sql.insideaag.com:1450' 'SafetyME' 'Shakennotstirred1!'
#isql -v DSN_NAME USERNAME PASSWORD




Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 13 for SQL Server' : file not found (0) (SQLDriverConnect)")

In [ ]:
query = 'SELECT * FROM [Safety].[dbo].[vw_ETQ_Events_fast]'
cursor.execute(query)

# Fetch data
data = cursor.fetchall()

In [ ]:
import pyodbc
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# Define your SQL Server connection parameters
server = 'your_server_name'
database = 'your_database_name'
username = 'your_username'
password = 'your_password'

# Define your Azure Blob Storage connection parameters
storage_account_name = 'your_storage_account_name'
storage_account_key = 'your_storage_account_key'
container_name = 'your_container_name'
blob_name = 'your_blob_name'

# Define your SQL query to fetch data
sql_query = 'SELECT * FROM your_table_name'

# Connect to SQL Server
connection_string = f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(connection_string)

# Create a cursor to execute SQL queries
cursor = conn.cursor()

# Execute the SQL query and fetch data
cursor.execute(sql_query)
data = cursor.fetchall()

# Close the database connection
conn.close()

# Connect to Azure Blob Storage
blob_service_client = BlobServiceClient(account_url=f'https://{storage_account_name}.blob.core.windows.net', credential=storage_account_key)

# Get a reference to the container
container_client = blob_service_client.get_container_client(container_name)

# Convert the data to a string or bytes, depending on your needs
# For example, you can convert it to a CSV string
csv_data = '\n'.join(','.join(map(str, row)) for row in data)

# Upload the data to Azure Blob Storage as a blob
blob_client = container_client.get_blob_client(blob_name)
blob_client.upload_blob(csv_data, overwrite=True)

print(f'Data uploaded to Azure Blob Storage: {blob_name}')


In [73]:
# # Semantic Hybrid Search
# query = "What is the definition of a Minor Accident?"

# query_embeddings = embeddings.embed_query(query)

# search_client_v2 = SearchClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, index_name=AZURE_SEARCH_INDEX_NAME, credential=credential)
# vector = Vector(value=query_embeddings, k=3, fields="content_vector")  

# results = search_client_v2.search(  
#     search_text=query,  
#     vectors=[vector],
#     select=["title", "content", "content_vector"],
#     query_type="semantic", query_language="en-us", semantic_configuration_name='my-semantic-config', query_caption="extractive", query_answer="extractive",
#     top=3
# )

# semantic_answers = results.get_answers()
# for answer in semantic_answers:
#     if answer.highlights:
#         print(f"Semantic Answer: {answer.highlights}")
#     else:
#         print(f"Semantic Answer: {answer.text}")
#     print(f"Semantic Answer Score: {answer.score}\n")


# for result in results:  
#     print(f"title: {result['title']}")  
#     print(f"Score: {result['@search.score']}")  
#     print(f"content: {result['content']}")  
#     print(f"page_number: {result['content_vector']}\n") 

#     captions = result["@search.captions"]
#     if captions:
#         caption = captions[0]
#         if caption.highlights:
#             print(f"Caption: {caption.highlights}\n")
#         else:
#             print(f"Caption: {caption.text}\n")
            
            

https://github.com/Alaska-ITS/Generative-AI-POC/blob/GenAI-Belete/Data-Ingestion/GenAI_Dcument_Indexer.py

https://techcommunity.microsoft.com/t5/azure-ai-services-blog/azure-cognitive-search-and-langchain-a-seamless-integration-for/ba-p/3901448

https://medium.com/mlearning-ai/using-langchain-vector-store-using-azure-cognitive-search-4ab89b626b92

https://python.langchain.com/docs/integrations/vectorstores/azuresearch

https://github.com/Azure/cognitive-search-vector-pr/blob/main/demo-python/code/azure-search-vector-python-sample.ipynb

https://github.com/Azure/cognitive-search-vector-pr/blob/main/demo-python/code/azure-search-vector-python-sample.ipynb